# 🧪 Rocket Propellants Deep Dive
## The Chemistry and Physics of Rocket Fuels

### What You'll Learn
1. Why "jet fuel" ≠ "rocket propellant"
2. The chemistry of combustion reactions
3. Why fuel choice matters (density vs efficiency)
4. How to calculate if your rocket reaches orbit

---

## 1. Rocket Propellants vs Jet Fuel

**Important distinction:**
- **Jet fuel** powers air-breathing engines (they use oxygen from air)
- **Rocket propellants** = Fuel + Oxidizer (rockets carry their own oxygen!)

A rocket must carry BOTH:
- **Fuel**: The stuff that burns (hydrogen, methane, kerosene)
- **Oxidizer**: The stuff it burns with (usually liquid oxygen, LOX)

This is why rockets are so heavy - they're carrying their own atmosphere!


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
plt.style.use('dark_background')

# Propellant database with real-world values
PROPELLANTS = {
    "Hydrolox (LH2/LOX)": {
        "isp_vac": 450, "isp_sl": 366,
        "density": 0.07,  # g/cm³ (very low!)
        "used_by": "Space Shuttle, SLS, Delta IV",
        "reaction": "2H₂ + O₂ → 2H₂O"
    },
    "Methalox (CH4/LOX)": {
        "isp_vac": 380, "isp_sl": 330,
        "density": 0.42,
        "used_by": "Starship, New Glenn, Vulcan",
        "reaction": "CH₄ + 2O₂ → CO₂ + 2H₂O"
    },
    "RP-1/LOX (Kerosene)": {
        "isp_vac": 350, "isp_sl": 282,
        "density": 0.81,
        "used_by": "Falcon 9, Saturn V, Atlas V",
        "reaction": "C₁₂H₂₆ + 18.5O₂ → 12CO₂ + 13H₂O"
    },
    "Solid (HTPB)": {
        "isp_vac": 280, "isp_sl": 242,
        "density": 1.80,
        "used_by": "Space Shuttle SRBs, Ariane boosters",
        "reaction": "(complex polymer combustion)"
    }
}

G0 = 9.80665  # Standard gravity

print("✅ Propellant database loaded!")
print("\nAvailable propellants:")
for name in PROPELLANTS.keys():
    print(f"  • {name}")


## 2. The Chemistry: Balancing Combustion Reactions

Every rocket engine is basically a controlled explosion. The fuel and oxidizer must mix in the **perfect ratio** for maximum energy.

### Example: Hydrogen Combustion

$$2H_2 + O_2 \rightarrow 2H_2O + \text{Energy}$$

- **Fuel Rich** (too much H₂): Some hydrogen doesn't burn. Wasted fuel, lower temperature.
- **Fuel Lean** (too much O₂): Burns TOO hot. Excess oxygen attacks metal → engine melts!
- **Perfect Balance**: Maximum energy release, clean water vapor exhaust.

### Why Real Engines Run Slightly Fuel-Rich
Engineers actually run engines a bit fuel-rich on purpose! The extra fuel:
1. Keeps temperatures manageable
2. Creates a protective gas layer on engine walls
3. Is less corrosive than excess oxygen


In [ ]:
# Compare all propellants
print("🚀 PROPELLANT COMPARISON")
print("=" * 75)
print(f"{'Propellant':<25} {'Isp (vac)':>10} {'Density':>10} {'Used By':<25}")
print("-" * 75)

for name, data in PROPELLANTS.items():
    short_name = name.split(" (")[0] if "(" in name else name
    print(f"{short_name:<25} {data['isp_vac']:>8} s {data['density']:>8.2f} {data['used_by']:<25}")

print("-" * 75)
print("\n💡 KEY INSIGHT: Higher Isp = more efficient, but often means lower density!")
print("   Hydrogen is 10x more efficient than kerosene... but needs 10x larger tanks.")


## 3. The Density Problem: Why Tank Size Matters

Look at this visualization - for the same MASS of propellant, hydrogen needs MUCH bigger tanks!


In [ ]:
# Visualize the trade-off
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

names = [p.split(" (")[0] for p in PROPELLANTS.keys()]
isp_values = [p["isp_vac"] for p in PROPELLANTS.values()]
density_values = [p["density"] for p in PROPELLANTS.values()]
colors = ['#1f77b4', '#9467bd', '#d62728', '#ff7f0e']

# Efficiency chart
ax1.barh(names, isp_values, color=colors)
ax1.set_xlabel('Specific Impulse (seconds)', fontsize=11)
ax1.set_title('🎯 Efficiency: Higher is Better', color='#00d4ff')
ax1.axvline(x=350, color='white', linestyle='--', alpha=0.3, label='LEO baseline')

# Density chart  
ax2.barh(names, density_values, color=colors)
ax2.set_xlabel('Density (g/cm³)', fontsize=11)
ax2.set_title('📦 Density: Higher = Smaller Tanks', color='#00d4ff')

plt.tight_layout()
plt.show()

print("\n🤔 THE GOLDILOCKS PROBLEM:")
print("   • Hydrogen: BEST efficiency... but tanks are HUGE (Space Shuttle's orange tank)")
print("   • Kerosene: Dense and powerful... but lower efficiency, leaves soot")
print("   • Methane: Just right! Good efficiency AND reasonable tank size")
print("\n   This is why SpaceX chose methane for Starship!")


## 4. Can Your Rocket Reach Orbit?

Let's use the Tsiolkovsky Rocket Equation to find out!

$$\Delta v = I_{sp} \cdot g_0 \cdot \ln\left(\frac{m_{initial}}{m_{final}}\right)$$

**Mission Requirements:**
| Destination | Delta-V Needed |
|-------------|----------------|
| Suborbital | ~2,000 m/s |
| LEO | ~9,400 m/s |
| Moon | ~12,500 m/s |
| Mars | ~16,000 m/s |


In [ ]:
def calculate_delta_v(propellant_name, dry_mass_kg, fuel_mass_kg):
    """Calculate delta-v using the Tsiolkovsky Rocket Equation."""
    isp = PROPELLANTS[propellant_name]["isp_vac"]
    total_mass = dry_mass_kg + fuel_mass_kg
    mass_ratio = total_mass / dry_mass_kg
    delta_v = isp * G0 * math.log(mass_ratio)
    return delta_v

# Test with a medium rocket
dry_mass = 10000    # 10 tons (rocket structure + payload)
fuel_mass = 90000   # 90 tons of propellant

print("🚀 ROCKET MISSION ANALYSIS")
print(f"Dry Mass: {dry_mass:,} kg | Fuel Mass: {fuel_mass:,} kg")
print(f"Fuel Fraction: {fuel_mass/(dry_mass+fuel_mass)*100:.1f}%")
print("=" * 60)

for name in PROPELLANTS.keys():
    dv = calculate_delta_v(name, dry_mass, fuel_mass)
    short_name = name.split(" (")[0]
    
    # Determine what mission is possible
    if dv >= 16000:
        mission = "✅ MARS!"
    elif dv >= 12500:
        mission = "🌙 Moon"
    elif dv >= 9400:
        mission = "🛰️ LEO"
    elif dv >= 2000:
        mission = "🚀 Suborbital"
    else:
        mission = "❌ Grounded"
    
    print(f"{short_name:<20} → {dv:>8,.0f} m/s  {mission}")


## 🎓 Key Takeaways

1. **Rockets carry their own oxygen** - Unlike jets, they work in the vacuum of space

2. **Isp = Efficiency** - Higher specific impulse means more delta-v per kg of fuel

3. **Density matters** - Hydrogen is efficient but needs huge tanks

4. **Methane is the future** - Best balance of efficiency, density, and reusability

5. **The rocket equation is cruel** - Doubling fuel doesn't double speed (logarithm!)

---

## 🚀 Try the Interactive App!

For a full interactive experience with sliders and visualizations:

```bash
cd ../Projects/Propellant_Explorer
pip install streamlit plotly pandas
streamlit run app.py
```

---

*"The rocket worked perfectly, except for landing on the wrong planet."* — Wernher von Braun (on early rocket failures)
